[Reference](https://levelup.gitconnected.com/overload-functions-in-python-d045375cff04)

In [1]:
def area(radius):
  return 3.14 * radius ** 2

locals()

{'In': ['', 'def area(radius):\n  return 3.14 * radius ** 2\n\nlocals()'],
 'Out': {},
 '_': '',
 '__': '',
 '___': '',
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__loader__': None,
 '__name__': '__main__',
 '__package__': None,
 '__spec__': None,
 '_dh': ['/content'],
 '_i': '',
 '_i1': 'def area(radius):\n  return 3.14 * radius ** 2\n\nlocals()',
 '_ih': ['', 'def area(radius):\n  return 3.14 * radius ** 2\n\nlocals()'],
 '_ii': '',
 '_iii': '',
 '_oh': {},
 '_sh': <module 'IPython.core.shadowns' from '/usr/local/lib/python3.6/dist-packages/IPython/core/shadowns.py'>,
 'area': <function __main__.area>,
 'exit': <IPython.core.autocall.ZMQExitAutocall at 0x7fbdb9322f28>,
 'get_ipython': <bound method InteractiveShell.get_ipython of <google.colab._shell.Shell object at 0x7fbdba59c9b0>>,
 'quit': <IPython.core.autocall.ZMQExitAutocall at 0x7fbdb9322f28>}

In [2]:
from inspect import getfullargspec

class Function(object):
  """Function is a wrap over standard python function.
  """

  def __init__(self, fn):
    self.fn = fn

  def __call__(self, *args, **kwargs):
    """when invoked like a function it internally invokes
    the wrapped function and returns the returned value.
    """
    return self.fn(*args, **kwargs)

  def key(self, args=None):
    """Returns the key that will uniquely identify
    a function (even when it is overloaded).
    """
    # if args not specified, extract the arguments from the
    # function definition
    if args is None:
      args = getfullargspec(self.fn).args
    return tuple([
      self.fn.__module__,
      self.fn.__class__,
      self.fn.__name__,
      len(args or []),
    ])

In [3]:
def area(l, b):
  return l * b

func = Function(area)
func.key()

('__main__', function, 'area', 2)

In [4]:
func(3, 4)

12

In [5]:
class Namespace(object):
  """Namespace is the singleton class that is responsible
  for holding all the functions.
  """

  __instance = None
  def __init__(self):
    if self.__instance is None:
      self.function_map = dict()
      Namespace.__instance = self
    else:
      raise Exception("cannot instantiate a virtual Namespace again")

  @staticmethod
  def get_instance():
    if Namespace.__instance is None:
      Namespace()
    return Namespace.__instance

  def register(self, fn):
    """registers the function in the virtual namespace and returns
    an instance of callable Function that wraps the
    function fn.
    """
    func = Function(fn)
    self.function_map[func.key()] = fn
    return func

In [6]:
def area(l, b):
  return l * b

In [7]:
namespace = Namespace.get_instance()
func = namespace.register(area)
func(3, 4)

12

In [8]:
import time


def my_decorator(fn):
  """my_decorator is a custom decorator that wraps any function
  and prints on stdout the time for execution.
  """

  def wrapper_function(*args, **kwargs):
    start_time = time.time()
    # invoking the wrapped function and getting the return value.
    value = fn(*args, **kwargs)
    print("the function execution took:", time.time() - start_time, "seconds")
    # returning the value got after invoking the wrapped function
    return value

  return wrapper_function


@my_decorator
def area(l, b):
  return l * b


area(3, 4)

the function execution took: 9.5367431640625e-07 seconds


12

In [9]:
def overload(fn):
  """overload is the decorator that wraps the function
  and returns a callable object of type Function.
  """
  return Namespace.get_instance().register(fn)

In [10]:
def get(self, fn, *args):
  """get returns the matching function from the virtual namespace.
  return None if it did not fund any matching function.
  """
  func = Function(fn)
  return self.function_map.get(func.key(args=args))

In [11]:
def __call__(self, *args, **kwargs):
  """Overriding the __call__ function which makes the
  instance callable.
  """
  
  # fetching the function to be invoked from the virtual namespace
  # through the arguments.
  fn = Namespace.get_instance().get(self.fn, *args)
  
  if not fn:
    raise Exception("no matching function found.")
  
  # invoking the wrapped function and returning the value.
  return fn(*args, **kwargs)

In [12]:
@overload
def area(l, b):
  return l * b


@overload
def area(r):
  import math
  return math.pi * r ** 2


area(3, 4)

TypeError: ignored

In [13]:
area(7)

153.93804002589985